# Búsqueda semántica de texto
En este notebook vamos a implementar un buscador semántico de textos similares mediante un modelo *Transformer* usando el pipeline de extracción de características de Hugging Face (https://huggingface.co/docs/transformers/v4.29.0/en/main_classes/pipelines#transformers.FeatureExtractionPipeline). \
Vamos a usar el conjunto de noticias del dataset Lee.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModel


In [ ]:
lee_data_file = 'lee_background.cor'

In [ ]:
#Leemos todas las noticias
#Al usar transformers podemos obviar el pre-procesado del texto
with open(lee_data_file) as f:
    docs = f.readlines()

In [ ]:
len(docs)

In [ ]:
display(docs[0])

Usamos el embedding del token `[CLS]`a la salida del modelo BERT en inferencia para extraer los vectores de documentos del corpus. Se puede probar con la `pooler_output`

In [ ]:
modelo = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def encode(doc):
  input = tokenizer(doc, truncation=True, return_tensors="pt")
  output = modelo(**input)

  return output.last_hidden_state[0,0,:].detach().numpy() #salida [CLS]
  #return output.pooler_output.detach().numpy().ravel() #salida 'pooler_output'


In [ ]:
output = encode(docs[0])

In [ ]:
output.shape

Devuelve una lista de tensores (por cada documento)

In [ ]:
from tqdm import tqdm
doc_embeddings=np.stack([encode(doc) for doc in tqdm(docs)]) #usamos el vector del token [CLS] como embedding de cada doc

In [ ]:
doc_embeddings.shape

También se podría haber usado el *pipeline* de extracción de características para extraer el embeddings `[CLS]` de cada texto, pero con la sobrecarga de extraer todos los embeddings de todas las capas internas del transformer:

In [ ]:
extractor = pipeline(model="bert-base-uncased", task="feature-extraction")
result = extractor(docs, 
    tokenize_kwargs={'padding':True,'truncation':True,'max_length':512},
    return_tensors=True)

doc_embeddings=np.stack([l[0,0,:].numpy() for l in tqdm(result)])

Los embeddings generados para cada documento son los que usaremos para calcular la similitud entre documentos (con la distancia coseno). Es lo que se conoce como técnica *Bi-encoder*:  
>A Bi-Encoder Sentence Transformer model takes in one text at a time as input and outputs a fixed dimension embedding vector as the output. We can then compare any two documents by computing the cosine similarity between the embeddings of those two documents.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#Vemos la similitud de todos los documentos con todos
sims = cosine_similarity(doc_embeddings, doc_embeddings)
sims.shape

Vemos la similitud del primer documento al resto

In [ ]:
sims[0, :]

In [ ]:
sims[0, :].shape

In [ ]:
#Ordenamos de mayor a menor
sims_sorted = sorted(enumerate(sims[0,:]), key=lambda item: -item[1])
print(sims_sorted[:10])

In [ ]:
#Noticia más cercana
display(docs[sims_sorted[1][0]])

In [ ]:
#5 noticias más similares
for idx, score in sims_sorted[1:6]:
        print(docs[idx], f"(Score: {score})" )

In [ ]:
#Creamos un texto nuevo y buscamos la noticia más similar
texto = """the new Pakistan government falled in the terrorist attack by the islamic group Hamas"""
texto_embedding = encode(texto)

In [ ]:
texto_embedding.reshape(1,-1).shape

In [ ]:
#Comparamos con el resto
sims = cosine_similarity(texto_embedding.reshape(1,-1), doc_embeddings)[0]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
sims.shape

In [ ]:
print(sims_sorted[:10])

In [ ]:
#5 noticias más similares
for idx, score in sims_sorted[0:5]:
        print(docs[idx], f"(Score: {score})" )